# H(z) Reconstrution via Gaussian Process (GP)

In [ ]:
"""from gaussian_process import GPReconstructionH
from plots_rec import HReconstructionPlot
from equations import FiducialModel
from obs_data import H_data

# Load your data
data_Hz = H_data()

z_values, H_obs, errors = data_Hz.H_z_data()

# Choose the GP parameters
gp_h = GPReconstructionH(z_values, H_obs, errors)
gp_h.optimize(num_restarts=10, verbose=False)
mean, var, mean_deriv, var_deriv = gp_h.predict()
z_val = gp_h.z_pred()

import numpy as np

# Combine os dados em um array de duas colunas
data_to_save = np.column_stack((z_val, mean))

# Salve no arquivo .txt
np.savetxt('data/H_rec_gp_data.txt', data_to_save, fmt='%.6f', header='z_val  mean', comments='')


fiducial_model = FiducialModel()

# Plot the figure
plotter = HReconstructionPlot(gp_h, fiducial_model)
plotter.plot('Figuras/H_reconstructed.png')"""

# H(z) Reconstrution via Artificial Neural Network (ANN): ReFANN code

In [ ]:
"""import refann as rf
import time
import numpy as np
import matplotlib.pyplot as plt

# Starting the ANN
start_time = time.time()

Hz = np.loadtxt('data/Hz35data.txt', skiprows=1)

rec = rf.ANN(Hz,mid_node=4096,hidden_layer=1,hp_model='rec_2')
rec.iteration = 30000
rec.train()
func = rec.predict(xpoint=np.linspace(0, 2, 201))
#func = rec.predict(xspace=(0, 2, 201)) #or use this
rec.save_func(path='data', obsName='Hz35') #save the reconstructed function

# rec.plot_loss()
rec.plot_func()
print ("Time elapsed: %.3f mins" %((time.time()-start_time)/60))
plt.show()"""

# Bayesian analysis and MCMC

## Defining priors

In [ ]:
from likelihood import JointPriors

# Defining the priors and the type of distribuition
param_configs_w_frb = {
    # 'Omega_bh2': ((0.01,0.05), 'uniform'), 
    'Omega_bh2': ((0.02235, 0.00049), 'gaussian'),
    'DM_host_0': ((55, 225), 'uniform')
}

param_configs_w_sne = {
    'H0': ((20, 100), 'uniform'),
    'Omega_m': ((0.1, 0.9), 'uniform')
}

P_frb = JointPriors(param_configs_w_frb)
P_sne = JointPriors(param_configs_w_sne)

### Models and parameters

In [ ]:
from likelihood import JointLikelihoodFunction
from equations import DM_EXT_model_Rec, Modulus_sne

model = DM_EXT_model_Rec()
mu_wCDM = Modulus_sne()

# Creating an instance of JointLikelihoodFunction
LF_gp_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, DM_host_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2,     
        DM_host_0=DM_host_0,
        rec_type='GP'
    ) 
    }
)

LF_ann_frb = JointLikelihoodFunction(
    { 'FRB': lambda z, Omega_bh2, DM_host_0: model.DM_ext_th(
        z=z,
        f_IGM=0.83,
        model_type='constant',
        Omega_bh2=Omega_bh2,     
        DM_host_0=DM_host_0,
        rec_type='ANN'
    ) 
    }
)

## Preparing the samples

### Analysis for 66 FRBs

In [ ]:
from obs_data import FRB_data, SNe_data
import ultranest

# Instantiate the FRB_data class for 66 FRBs
frb_data = FRB_data(n_frb=66)

# Call the select_data method to get the observed data
z_frb, DM_ext, sigma_ext = frb_data.select_data()

"""sne_data_union = SNe_data(sample_sne='Union3')

z_sne_union, mu_sne_union, cov_matrix_inv_union = sne_data_union.load_data()

sne_data_pantheon = SNe_data(sample_sne='Pantheon+SH0ES')

z_sne, z_alt, mu_sne, cov_matrix_inv = sne_data_pantheon.load_data()"""

# Configuring the ultranest samplers
sampler_gp_frb = ultranest.ReactiveNestedSampler(
    P_frb.param_names,
    lambda params: LF_gp_frb.log_likelihood(
        dict(zip(P_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_frb.prior_transform
)

sampler_ann_frb = ultranest.ReactiveNestedSampler(
    P_frb.param_names,
    lambda params: LF_ann_frb.log_likelihood(
        dict(zip(P_frb.param_names, params)),
        {
            'FRB': (z_frb, DM_ext, sigma_ext)
        }
    ),
    P_frb.prior_transform
)

In [ ]:
gp_frb = sampler_gp_frb.run(min_num_live_points=400)
sampler_gp_frb.print_results()

In [ ]:
ann_frb = sampler_ann_frb.run(min_num_live_points=400)
sampler_ann_frb.print_results()

In [ ]:
from getdist import plots, MCSamples
import numpy as np

# Extraindo amostras dos resultados para frb
samples_gp_frb = gp_frb['samples']
samples_ann_frb = ann_frb['samples']

labels1 = ['\\Omega_{b}h^2', 'DM_{host,0}']
names1 = P_frb.param_names

# Calculando o parâmetro derivado
derived_param_gp = 100 * np.sqrt(samples_gp_frb[:, 0] / 0.0493)
derived_param_ann = 100 * np.sqrt(samples_ann_frb[:, 0] / 0.0493)

# Adicionando o parâmetro derivado nas amostras
samples_with_derived_gp = np.hstack([samples_gp_frb, derived_param_gp[:, None]])
samples_with_derived_ann = np.hstack([samples_ann_frb, derived_param_ann[:, None]])

names_extended = names1 + ['H0']
labels_extended = ['\\Omega_{b}h^2', 'DM_{host,0}', 'H_0']

# Criando MCSamples com o parâmetro derivado
mcsamples_gp_frb = MCSamples(samples=samples_with_derived_gp, 
                           names=names_extended, 
                           labels=labels_extended)

mcsamples_ann_frb = MCSamples(samples=samples_with_derived_ann, 
                           names=names_extended, 
                           labels=labels_extended)

In [ ]:
# Plotando os Triangle plots
g = plots.get_subplot_plotter()
mcsamples_ann_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
mcsamples_gp_frb.updateSettings({'smooth_scale_2D': 0.9, 'smooth_scale_1D': 0.9})
g.settings.num_plot_contours = 2
g.triangle_plot([mcsamples_gp_frb, mcsamples_ann_frb], names1, filled=True, contour_colors=['blue', 'red'], 
                legend_labels=['GaPP', 'ReFANN'])
g.export('Figuras/rec_frb.png', dpi=600)

In [ ]:
import pandas as pd

# Dicionário de amostras e nomes de arquivos
amostras = {
    "frb_gapp": samples_gp_frb,
    "frb_refann": samples_ann_frb
}

# Salva cada conjunto de amostras em arquivos CSV
for nome, dados in amostras.items():
    df = pd.DataFrame(dados)
    df.to_csv(f"Samplers/{nome}.csv", index=False)


In [ ]:
"""import pandas as pd
df1_frb = pd.read_csv("Samplers/samples1_frb.csv")
df1_sne = pd.read_csv("Samplers/samples1_sne.csv")
df1_sne_frb = pd.read_csv("Samplers/samples1_sne_frb.csv")
df2_frb = pd.read_csv("Samplers/samples2_frb.csv")
df2_sne = pd.read_csv("Samplers/samples2_sne.csv")
df2_sne_frb = pd.read_csv("Samplers/samples2_sne_frb.csv")"""

## Model Comparison

In [ ]:
"""samples = {
    "Sample_frb": {
        "results": {
            'wCDM': {'logz': wCDM_frb['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_frb['logz'], 'num_params': 6},
        }
    },
    "Sample_sne": {
        "results": {
            'wCDM': {'logz': wCDM_sne['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne['logz'], 'num_params': 6},
        }
    },
    "Sample_sne_frb": {
        "results": {
            'wCDM': {'logz': wCDM_sne_frb['logz'], 'num_params': 5},
            'CPL': {'logz': CPL_sne_frb['logz'], 'num_params': 6},
        }
    }
}"""

In [ ]:
"""from bayesian_analysis import ModelComparison

comparison = ModelComparison(samples)
comparison.run_comparisons(save_to_file='comparisons_output_frb+SNe.txt')"""

In [ ]:
from bayesian_analysis import SaveResults

# Initialize the class with the desired output file
results_saver = SaveResults("Results_mcmc/results_frb_rec_Hz.txt")

# Optional: Clear the file if you want to start fresh
results_saver.reset_file()

results_saver.save_to_txt(mcsamples_gp_frb, 'GaPP')
results_saver.save_to_txt(mcsamples_ann_frb, 'Refann')

In [ ]:
from bayesian_analysis import CompareMCMCResults

comparator = CompareMCMCResults('Results_mcmc/mcmc_comparison_results.txt')

comparator.reset_file()

# Faz a comparação entre os dois modelos
comparator.compare_errors(mcsamples_gp_frb, mcsamples_ann_frb, "GaPP", "Refann")

In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_16, y=dm_ext_obs_16, fmt='o', alpha=0.6, color='red', label='16 FRBs', ms=2)

z_test = np.linspace(0, 1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0, omega_a  in sampler_p2_16.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,  
        omega_a=omega_a,
        cosmo_type='non_standard',
        param_type='CPL'))

band.line(color='k', linestyle='-', label='CPL parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_16.png', format='png', dpi=600)"""

In [ ]:
"""from ultranest.plot import PredictionBand
from equations import DM_EXT_model
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.xlabel('$z$')
plt.ylabel('$DM_{ext}(z)$')
plt.errorbar(x=z_values_63, y=dm_ext_obs_63, fmt='o', alpha=0.6, color='red', label='63 FRBs', ms=2)

z_test = np.linspace(0, 1.1, 100)

band = PredictionBand(z_test)
model_fit = DM_EXT_model()
# go through the solutions
for H_0, A, beta, omega_0  in sampler_constant_63.results['samples']:
    # compute for each time the y value
    band.add(model_fit.DM_ext_th(z=z_test,
        f_IGM=0.83,
        model_type='constant',
        Omega_b=None,  
        Omega_m=None,     
        H_today=H_0,
        A=A,
        beta=beta,
        omega_0=omega_0,
        cosmo_type='non_standard',
        param_type='constant')
    )

band.line(color='k', linestyle='-', label='Constant parameterization', linewidth=1.5)
# add 1 sigma quantile
band.shade(color='green', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='green', alpha=0.2)

plt.legend()
plt.savefig('Figuras/DM_ext_bestfit_63.png', format='png', dpi=600)"""